In [1]:
import sys
sys.path.append('../')
import fm
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [2]:
def pow(x):
  return x ** 2

In [3]:
pow_traced = symbolic_trace(pow)

In [18]:
interp = utils.ProfilingInterpreter(pow_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long))
print(interp.summary(True))

total true time 11.011362075805664 ms
total time: 11.199712753295898 ms
Op type        Op        Average runtime (ms)    Pct total runtime
-------------  ------  ----------------------  -------------------
call_function  pow_1               10.9253               97.5498
output         output               0.0503063             0.449175
placeholder    x                    0.0357628             0.319319


In [38]:
output_onnx = 'model_ori.onnx'
torch.onnx.export(pow_traced,               # 模型 being run
                  torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  output_onnx,        # where to save the model (can be a file or file-like object)
)

In [34]:
def pow_modify(x):
  y = torch.empty((4096,100,32))
  y[:,:50,:] = x[0,:50,:] ** 2
  y[:,50:,:] = x[:,50:,:] ** 2
  return y

In [31]:
output_onnx = 'model.onnx'


In [35]:
pow_modify_traced = symbolic_trace(pow_modify)

In [36]:
torch.onnx.export(pow_modify_traced,               # 模型 being run
                  torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  output_onnx,        # where to save the model (can be a file or file-like object)
)

In [37]:
import onnx
from onnxsim import simplify

# 加载ONNX模型
model_path = output_onnx  # 替换为你的ONNX模型路径
original_model = onnx.load(model_path)

# 简化模型
simplified_model, check = simplify(original_model)

# 检查简化是否成功
assert check, "Simplified ONNX model could not be validated"

# 保存简化后的模型
simplified_model_path = output_onnx  # 替换为简化后模型的保存路径
onnx.save(simplified_model, simplified_model_path)

print(f"Simplified model has been saved to {simplified_model_path}")


Simplified model has been saved to model.onnx


In [10]:
pow_modify_traced.print_readable()

class pow_modify(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        getitem = x[(0, slice(None, 50, None), slice(None, None, None))]
        pow_1 = getitem ** 2;  getitem = None
        _tensor_constant0 = self._tensor_constant0
        setitem = _tensor_constant0.__setitem__((slice(None, None, None), slice(None, 50, None), slice(None, None, None)), pow_1);  _tensor_constant0 = pow_1 = None
        getitem_1 = x[(slice(None, None, None), slice(50, None, None), slice(None, None, None))];  x = None
        pow_2 = getitem_1 ** 2;  getitem_1 = None
        _tensor_constant0_1 = self._tensor_constant0
        setitem_1 = _tensor_constant0_1.__setitem__((slice(None, None, None), slice(50, None, None), slice(None, None, None)), pow_2);  _tensor_constant0_1 = pow_2 = None
        _tensor_constant0_2 = self._tensor_constant0
        return _tensor_constant0_2
        


'class pow_modify(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        getitem = x[(0, slice(None, 50, None), slice(None, None, None))]\n        pow_1 = getitem ** 2;  getitem = None\n        _tensor_constant0 = self._tensor_constant0\n        setitem = _tensor_constant0.__setitem__((slice(None, None, None), slice(None, 50, None), slice(None, None, None)), pow_1);  _tensor_constant0 = pow_1 = None\n        getitem_1 = x[(slice(None, None, None), slice(50, None, None), slice(None, None, None))];  x = None\n        pow_2 = getitem_1 ** 2;  getitem_1 = None\n        _tensor_constant0_1 = self._tensor_constant0\n        setitem_1 = _tensor_constant0_1.__setitem__((slice(None, None, None), slice(50, None, None), slice(None, None, None)), pow_2);  _tensor_constant0_1 = pow_2 = None\n        _tensor_constant0_2 = self._tensor_constant0\n        return _tensor_constant0_2\n        '

In [22]:
interp = utils.ProfilingInterpreter(pow_modify_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long))
print(interp.summary(True))

total true time 7.445096969604492 ms
total time: 11.996269226074219 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  pow_2                             5.27978             44.0118
call_method    setitem_1                         0.944138             7.87026
call_method    setitem                           0.827789             6.90039
call_function  getitem                           0.0925064            0.771126
call_function  getitem_1                         0.0760555            0.633993
call_function  pow_1                             0.0598431            0.498847
get_attr       _tensor_constant0_1               0.0522137            0.435249
get_attr       _tensor_constant0_2               0.0474453            0.3955
placeholder    x                                 0.0348091            0.290166
get_attr       _tensor_constant0                 0.020504       

In [24]:
def pow_modify1(x):
  x0 = x[0,:50,:] ** 2
  x1 = x[:,50:,:] ** 2
  x0 = x0.repeat(4096,1,1)
  
  return torch.concat([x0,x1], dim = 1)

In [26]:
pow_modify_traced1 = symbolic_trace(pow_modify1)

In [ ]:
output_onnx = 'model_concat.onnx'


In [47]:
torch.onnx.export(pow_modify_traced1,               # 模型 being run
                  torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  output_onnx,        # where to save the model (can be a file or file-like object)
)

In [28]:
interp = utils.ProfilingInterpreter(pow_modify_traced1)
interp.run(torch.randint(low=0, high=88, size=(4096,100,32), dtype=torch.long))
print(interp.summary(True))

total true time 18.984079360961914 ms
total time: 27.439594268798828 ms
Op type        Op           Average runtime (ms)    Pct total runtime
-------------  ---------  ----------------------  -------------------
call_function  concat                  6.90866              25.1777
call_method    repeat                  6.0451               22.0306
call_function  pow_2                   5.76925              21.0253
call_function  getitem                 0.0889301             0.324094
call_function  pow_1                   0.0545979             0.198975
output         output                  0.0545979             0.198975
placeholder    x                       0.0360012             0.131202
call_function  getitem_1               0.0269413             0.098184


In [39]:
import onnxruntime as ort


In [40]:
session = ort.InferenceSession('model_ori.onnx')


In [42]:
%timeit -n 1 -r 30 session.run(None,{'x' : np.random.randint(low=0, high=88, size=(4096, 100,32), dtype=np.int64)})

183 ms ± 4.06 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [43]:
session = ort.InferenceSession('model.onnx')


In [45]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Gather_0' : np.random.randint(low=0, high=88, size=(4096, 100,32), dtype=np.int64)})

351 ms ± 13.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [50]:
import onnx
from onnxsim import simplify

# 加载ONNX模型
model_path = output_onnx  # 替换为你的ONNX模型路径
original_model = onnx.load(model_path)

# 简化模型
simplified_model, check = simplify(original_model)

# 检查简化是否成功
assert check, "Simplified ONNX model could not be validated"

# 保存简化后的模型
simplified_model_path = output_onnx  # 替换为简化后模型的保存路径
onnx.save(simplified_model, simplified_model_path)

print(f"Simplified model has been saved to {simplified_model_path}")

Simplified model has been saved to model_concat.onnx


In [51]:
session = ort.InferenceSession('model_concat.onnx')


In [52]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Gather_0' : np.random.randint(low=0, high=88, size=(4096, 100,32), dtype=np.int64)})

214 ms ± 17.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [55]:
class testModel(torch.nn.Module):
  def __init__(self,):
    super().__init__()
    self.linear = nn.Linear(in_features=400,out_features=400)
  
  def forward(self, x):
    return self.linear(x)

In [56]:
model = testModel()
torch.onnx.export(model,torch.ones((1024,1,200,400)),'testmodel.onnx')

In [ ]:
def inner_product(x):
  square_of_sum = torch.sum(x, dim=1) ** 2
  sum_of_square = torch.sum(x ** 2, dim=1)
  ix = square_of_sum - sum_of_square
  return 0.5 * ix

In [ ]:
def inner_product(x):
  sum_of_x = torch.sum(x, dim = 1)
  square_of_sum = torch.pow(sum_of_x, 2)
  square_of_x = torch.pow(x, 2)
  sum_of_square = torch.sum(square_of_x, dim = 1)
  ix = torch.sub(square_of_sum, sum_of_square)
  return torch.mul(ix, 0.5)